### Set up LLM and Embedding

In [7]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
#from llama_index.llms.azure_openai import AzureOpenAI

# Using OpenAI as LLM with LlamaIndex
from llama_index.llms.openai import OpenAI

from llama_index.embeddings.openai import OpenAIEmbedding
#from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for OpenAi or AzureOpenAI. Replace with your own key
api_key = os.environ.get("OPENAI_API_KEY")

#Azure Endpoint URL. Replace with your own URL
#azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

#Note : Function calling support only available in GPT-4+ (maybe also in GPT-3.5 Turbo)
# Create the LLM object for OpenAI
llm = OpenAI(
    model="gpt-4o",  # or "gpt-4-turbo", "gpt-3.5-turbo-16k", etc.
    api_key=api_key
)

#Create the LLM object for Azure OpenAI
#llm=AzureOpenAI(
#    model="gpt-4o",
#    deployment_name="gpt-4o",
#    api_key=api_key,
#    azure_endpoint=azure_endpoint,
#    api_version="2024-08-01-preview",
#)

# Optionally, set as default for LlamaIndex
#Settings.llm = llm

#Set up the embedding model for RAG using OpenAI
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-large",    # or "text-embedding-3-large"
    api_key=api_key                    # your OpenAI API key (sk-...)
)

#Set up the embedding model for RAG using Azure OpenAI
#Settings.embed_model= AzureOpenAIEmbedding(
#    model="text-embedding-3-large",
#    deployment_name="text-embedding-3-large",
#    api_key=api_key,
#    azure_endpoint=azure_endpoint,
#    api_version="2023-05-15",
#)

### 02.03. Set up wikipedia tools

In [8]:
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

# Get list of wikipedia tools
wiki_spec = WikipediaToolSpec()
tool = wiki_spec.to_tool_list()[1]
wiki_tools = LoadAndSearchToolSpec.from_defaults(tool).to_tool_list()

#Review the list of tools
for tool in wiki_tools:
    print( "--------\n",tool.metadata.name, tool.metadata.description)

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    


### 02.04. Set up RAG for medications

In [9]:
#from llama_index.core import SimpleDirectoryReader
#from llama_index.readers.file import CSVReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.readers.json import JSONReader

medication_docs = JSONReader().load_data(
        input_file="datasets/Medications and dosages.json"
)
#Split the document
splitter=SentenceSplitter(chunk_size=200)
medication_nodes=splitter.get_nodes_from_documents(medication_docs)

#Index the document in memory
medication_index=VectorStoreIndex(medication_nodes)

#Create a query engine tool based on the CSV file
medication_query_engine = medication_index.as_query_engine()
medication_tool = QueryEngineTool.from_defaults(
    query_engine=medication_query_engine,
    description=(
        "Provides list of popular medications, diseases they are used to treat for and recommended dosages"
    ),
)

### 02.05. Set up & execute the Healthcare Agent

In [13]:
from llama_index.core.agent.react.base import ReActAgent
#Consolidate list of tools
healthcare_tools =wiki_tools + [medication_tool]

context = """
You are a healthcare assistant who can answer questions regarding 
diseases, their symptoms and medications for them.

Use the wikipedia tools read_search_data and search_data to answer questions about symptoms and possible 
diseases associated with those symptoms. 
Use the medication tool to find list of medications and dosages for a given
disease.

Use only the tools provided to answer questions and NOT your own memory.

"""

#Create a pre-built ReAct agent
healthcare_agent = ReActAgent.from_tools(
                healthcare_tools,
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


In [14]:
response=healthcare_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 2732d9a4-484c-4e8b-b76d-d5d35a2539e8. Step input: Which medication should i take for arthritis?
Thought: The user is asking about medications for arthritis. I can use the query_engine_tool to provide a list of medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'arthritis'}
Observation: Methotrexate is used for arthritis.
> Running step 4ed21736-042e-4283-a0db-0e6c3d2583b6. Step input: None
Thought: I have found information about Methotrexate being used for arthritis. Now, I need to provide the user with more details about this medication.
Action: query_engine_tool
Action Input: {'input': 'Methotrexate'}
Observation: The typical dosage for Methotrexate is 500-2000 mg daily, divided into doses. It is used for treating rheumatoid arthritis and cancer.
> Running step 9608c150-e53f-4347-9154-c0bbcbb6d03d. Step input: None
Thought: I have gathered information about Methotrexate, including its typical dosage and the conditions it is used to treat. Now,

In [17]:
response=healthcare_agent.chat("Which medication should i take for hyperthyroidism?")
print("*******\n Response : ",response.response)

> Running step 3683ae51-f86b-4ce8-a8d0-c89c30e63b44. Step input: Which medication should i take for hyperthyroidism?
Thought: The user is asking about medication for hyperthyroidism, which is a condition where the thyroid gland produces too much thyroid hormone. I can use the query engine tool to provide a list of medications commonly used to treat hyperthyroidism.
Action: query_engine_tool
Action Input: {'input': 'medications for hyperthyroidism'}
Observation: Levothyroxine
> Running step 8312dc2b-5e5a-4a80-bdba-780b2818af83. Step input: None
Thought: The user has mentioned "Levothyroxine" as a medication for hyperthyroidism. However, Levothyroxine is typically used to treat hypothyroidism, not hyperthyroidism. I need to provide the correct information regarding medications for hyperthyroidism.
Action: search_data
Action Input: {'query': 'medications for hyperthyroidism'}
Observation: Content loaded! You can now search the information using read_search_data
> Running step f4773ad1-f0e